In [1]:
#Импорты
import pandas as pd
from datetime import datetime as dt
from config import *

In [2]:
#Открытие CSV
def read_files(file_list, separator):
    res = {}
    for key, val in file_list.items():
        res[key] = pd.read_csv(filepath_or_buffer = val, sep = separator, header=0)

    return res

In [6]:
#Строку с датой в timestamp
def day(str_date: str) -> int:
    template = "%d.%m.%Y %H:%M:%S" if len(str_date) == 19 else "%Y-%m-%d %H:%M"
    return dt.strptime(str_date, template).timetuple().tm_yday

def income(str_income: str) -> int:
    res = str_income.split(",")[0]
    res = "".join(res.split())
    return res

def cash(str_cash: str) -> int:
    return 0 if str_cash == "0" else 1

# Функция очистки данных
def clean_pd(data: pd.DataFrame) -> pd.DataFrame:

    res = data[data["Тип документа"] == "Кассовый чек"].copy()
    f_columns = ["Сумма наличными, руб.", "Дата и время приема чека ОФД (UTC)", "Сумма документа, руб."]
    res = res[f_columns].copy()
    
    res.insert(0, column="cash", value=res.apply(lambda row: cash(row["Сумма наличными, руб."]), axis=1).astype("int64"))
    res.insert(0, column="day", value=res.apply(lambda row: day(row["Дата и время приема чека ОФД (UTC)"]), axis=1).astype("int64"))
    res.insert(0, column="income", value=res.apply(lambda row: income(row["Сумма документа, руб."]), axis=1).astype("int64"))

    res.drop(columns=f_columns, inplace=True)
    res.sort_values('day', ignore_index=True, inplace=True)
    
    return res

def group_pd(data: pd.DataFrame) -> pd.DataFrame:
    return data.groupby('day').sum().reset_index()

#Очистка данных
def prepair_data(data: dict[str: pd.DataFrame]) -> dict[str: pd.DataFrame]:
    res = {}
    for key, val in data.items():
        res[key] = clean_pd(val.copy())
        res[key] = group_pd(res[key])
        
    return res

In [8]:
file_list = {'2023': f'{DATA_PATH['raw']}ofd_2023.csv', '2024': f'{DATA_PATH['raw']}ofd_2024.csv'}
data = read_files(file_list, ";")
prepair_data = prepair_data(data)

In [9]:
for key, val in prepair_data.items():
    val.to_csv(f'{DATA_PATH['parsed']}income_{key}.csv', index=False)